In [1]:
import os

import pandas as pd
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer

from definitions import LOGS_DIR, ROOT_DIR

In [2]:
# Nicer log names
mapping = {
    'BPI_Challenge_2013_closed_problems.xes.gz': 'BPI13 Closed',
    'BPI_Challenge_2012.xes.gz': 'BPI12',
    'BPI_Challenge_2013_incidents.xes.gz': 'BPI13 Incidents',
    'BPI_Challenge_2013_open_problems.xes.gz': 'BPI13 Open',
    'BPI%20Challenge%202017.xes.gz': 'BPI17',
    'BPIC15_1.xes': 'BPI15',
    'Road_Traffic_Fine_Management_Process.xes.gz': 'RTFMP',
    'Sepsis%20Cases%20-%20Event%20Log.xes.gz': 'Sepsis',
    'helpdesk.csv': 'Helpdesk',
    'RTFMP_filtered.xes': 'RTFMP filtered',
}

In [3]:
def extract_stats(log):
    event_frequency_distribution = {}
    trace_length_distribution = {}
    longest_trace_length = 0
    amount_of_padding = 0
    for trace in log:
        # Update longest trace length
        if len(trace) > longest_trace_length:
            longest_trace_length = len(trace)
        
        # Save trace length distribution
        if len(trace) not in trace_length_distribution:
            trace_length_distribution[len(trace)] = 1
        else:
            trace_length_distribution[len(trace)] += 1
            
        # Save event frequency distribution
        for event in trace:
            if str(event['concept:name']) not in event_frequency_distribution:
                event_frequency_distribution[str(event['concept:name'])] = 1
            else:
                event_frequency_distribution[str(event['concept:name'])] += 1
    # Calculate amount of padding
    for trace in log:
        amount_of_padding += longest_trace_length - len(trace)
        
    return trace_length_distribution, event_frequency_distribution, amount_of_padding

log_stats = []
for file_name in sorted(os.listdir(LOGS_DIR)):
    if file_name.endswith('.xes') or file_name.endswith('.xes.gz'):
        xes_file_name = os.path.join(LOGS_DIR, file_name)
        log = xes_importer.apply(xes_file_name)
    elif file_name.endswith('.csv'):
        log = pm4py.format_dataframe(pd.read_csv(os.path.join(LOGS_DIR, file_name), sep=','),
                                     case_id='CaseID',
                                     activity_key='ActivityID',
                                     timestamp_key='CompleteTimestamp')
        log = pm4py.convert_to_event_log(log)
    else:
        raise Exception('Unknown file type')
    # Extract stats
    tld, efd, padding = extract_stats(log)
    # Save stats
    if str(file_name) in mapping:
        log_stats.append({
            'file_name': str(file_name),
            'trace_length_distribution': tld,
            'event_frequency_distribution': efd,
            'amount_of_padding': padding,
        })

parsing log, completed traces ::   0%|          | 0/43809 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
# transform to dataframe
log_stats_rows = []
for log, tld, efd, padding in log_stats:
    log_stats_rows.append({
        'log': mapping[log],
        'num_traces': sum(tld.values()),
        'num_events': sum(efd.values()),
        'num_activities': len(efd),
        'longest_trace_length': max(tld.keys()),
        'avg_trace_length': sum([k * v for k, v in tld.items()]) / sum(tld.values()),
        'shortest_trace_length': min(tld.keys()),
        'amount_of_padding': padding,
        'relative_padding': padding / (sum(efd.values()) + padding),
    })
df = pd.DataFrame(log_stats_rows)
df

In [22]:
def to_latex_table(df, filename, caption=None, label=None):
    style = df.style.format_index("\\textbf{{{}}}", escape="latex", axis=1) \
        .format(lambda x: str(x).replace('set()', '{}').replace("{", '\\{').replace("}", '\\}').replace("'", ''))

    col_format = 'l' + 'c' * (len(df.columns) - 1)
    output = style.to_latex(hrules=False, caption=caption, label=label, column_format=col_format)
    output = output.replace(r'\begin{tabular}{' + col_format + '}', r"""
    \centering
    \begin{NiceTabular}{""" + col_format + r"""}
    \CodeBefore
    \rowcolor{gray!50}{1}
    \rowcolors{2}{gray!25}{white}
    \Body""")
    output = output.replace(r'\end{tabular}', r'\end{NiceTabular}')

    if filename:
        with open(os.path.join(ROOT_DIR, 'reports', filename), 'w') as f:
            f.write(output)

    return output

In [25]:
to_latex_table(df, 'log_stats.tex', caption='Log statistics', label='tab:log_stats')

'\\begin{table}\n\\caption{Log statistics}\n\\label{tab:log_stats}\n\n    \\centering\n    \\begin{NiceTabular}{lccccccccc}\n    \\CodeBefore\n    \\rowcolor{gray!50}{1}\n    \\rowcolors{2}{gray!25}{white}\n    \\Body\n & \\textbf{file\\_name} & \\textbf{vocab\\_size} & \\textbf{num\\_traces} & \\textbf{num\\_events} & \\textbf{max\\_trace\\_length} & \\textbf{min\\_trace\\_length} & \\textbf{avg\\_trace\\_length} & \\textbf{num\\_events\\_copy} & \\textbf{amount\\_of\\_padding} & \\textbf{relative\\_padding} \\\\\n0 & BPI Challenge 2018.xes.gz & 42 & 43809 & 130244157 & 2973 & 24 & 57.39154055102833 & 2514266.0 & 127729891.0 & 0.980695748216943 \\\\\n1 & BPI%20Challenge%202017.xes.gz & 27 & 31509 & 5671620 & 180 & 10 & 38.15630454790695 & 1202267.0 & 4469353.0 & 0.7880205302894059 \\\\\n2 & BPIC15_1.xes & 399 & 1199 & 121099 & 101 & 2 & 43.55045871559633 & 52217.0 & 68882.0 & 0.5688073394495413 \\\\\n3 & BPI_Challenge_2012.xes.gz & 25 & 13087 & 2290225 & 175 & 3 & 20.035149384885763 &

In [51]:
name = 'Road_Traffic_Fine_Management_Process.xes.gz'
log = pm4py.read_xes(file_path=os.path.join(LOGS_DIR, name))

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

In [71]:
longest_trace = log.groupby('case:concept:name')['concept:name'].count().max()
longest_trace

AttributeError: 'EventLog' object has no attribute 'groupby'

In [53]:
variants = pm4py.get_variants(log)
variant_count = len(variants)
variant_count

231

In [63]:
variants_only_once = list(filter(lambda x: x[1] == 1, variants.items()))
variants_only_once = list(map(lambda x: x[0], variants_only_once))
variants_only_once

[('Create Fine',
  'Send Fine',
  'Insert Fine Notification',
  'Add penalty',
  'Insert Date Appeal to Prefecture',
  'Payment',
  'Receive Result Appeal from Prefecture',
  'Payment'),
 ('Create Fine',
  'Send Fine',
  'Insert Fine Notification',
  'Add penalty',
  'Notify Result Appeal to Offender',
  'Receive Result Appeal from Prefecture',
  'Send for Credit Collection'),
 ('Create Fine',
  'Send Fine',
  'Insert Fine Notification',
  'Insert Date Appeal to Prefecture',
  'Send Appeal to Prefecture',
  'Receive Result Appeal from Prefecture',
  'Add penalty',
  'Notify Result Appeal to Offender',
  'Appeal to Judge',
  'Send for Credit Collection'),
 ('Create Fine',
  'Send Fine',
  'Insert Date Appeal to Prefecture',
  'Insert Fine Notification',
  'Add penalty',
  'Send Appeal to Prefecture',
  'Receive Result Appeal from Prefecture',
  'Notify Result Appeal to Offender',
  'Send for Credit Collection'),
 ('Create Fine',
  'Send Fine',
  'Insert Fine Notification',
  'Receive Re

In [64]:
filtered2 = pm4py.filter_variants(log, variants_only_once, retain=False)
filtered2_variant_count = len(pm4py.get_variants(filtered2))
filtered2_variant_count

131

In [65]:
filtered2_longest_trace = filtered2.groupby('case:concept:name')['concept:name'].count().max()
filtered2_longest_trace

14

In [66]:
pm4py.write_xes(filtered2, os.path.join(LOGS_DIR, 'RTFMP_filtered.xes'))

exporting log, completed traces ::   0%|          | 0/150270 [00:00<?, ?it/s]

In [68]:
filtered2.attributes

AttributeError: 'DataFrame' object has no attribute 'attributes'

In [73]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter

log = xes_importer.apply(os.path.join(LOGS_DIR, name))

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

In [74]:
log.attributes

{'meta_org:different_resources_standard_deviation': 0.061,
 'meta_org:different_resources_min': 2,
 'meta_3TU:language': 'eng',
 'meta_concept:named_events_total': {'value': 561470,
  'children': {'Payment': 77601,
   'Insert Date Appeal to Prefecture': 4188,
   'Receive Result Appeal from Prefecture': 999,
   'Send Appeal to Prefecture': 4141,
   'Notify Result Appeal to Offender': 896,
   'Appeal to Judge': 555,
   'Add penalty': 79860,
   'Insert Fine Notification': 79860,
   'Send for Credit Collection': 59013,
   'Create Fine': 150370,
   'Send Fine': 103987}},
 'meta_org:group_events_average': {'value': 3.734,
  'children': {'UNKNOWN': 3.734}},
 'meta_org:different_groups_max': 1,
 'meta_life:different_transitions_min': 1,
 'meta_concept:different_names_standard_deviation': 1.578,
 'meta_time:log_duration': 424825200.0,
 'meta_3TU:creation_place': 'Eindhoven',
 'meta_general:classifiers': {'value': 1,
  'children': {'Event Name': {'value': 'concept:name',
    'children': {'meta_g

In [75]:
variants = pm4py.get_variants(log)
variant_count = len(variants)
variant_count

C:\Users\Hannes\anaconda3\envs\transfer_learning\lib\site-packages\pm4py\utils.py:508: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


231

In [80]:
variants_only_once = list(filter(lambda x: len(x[1]) == 1, variants.items()))
variants_only_once = list(map(lambda x: x[0], variants_only_once))
len(variants_only_once)

100

In [81]:
filtered2 = pm4py.filter_variants(log, variants_only_once, retain=False)
filtered2_variant_count = len(pm4py.get_variants(filtered2))
filtered2_variant_count

C:\Users\Hannes\anaconda3\envs\transfer_learning\lib\site-packages\pm4py\utils.py:508: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


131

In [82]:
xes_exporter.apply(filtered2, os.path.join(LOGS_DIR, 'RTFMP_filtered.xes'))

exporting log, completed traces ::   0%|          | 0/150270 [00:00<?, ?it/s]

In [83]:
abc = xes_importer.apply(os.path.join(LOGS_DIR, 'RTFMP_filtered.xes'))
abc.attributes['concept:name']

parsing log, completed traces ::   0%|          | 0/150270 [00:00<?, ?it/s]

'Road Traffic Fine Management Process'